In [54]:
import boto3
import json
import base64
import time
import re
import os
from botocore.exceptions import ClientError
import logging
import redis
from datetime import datetime , date
from insights import construct_call_conversation , enrollment_prompt_generator , load_claude2 , sns_data_postprocessing  
from insights import sns_publisher , get_insights_prompt , get_enrollment_prompt , insights_prompt_generator

today_date = date.today()

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name= os.environ.get("Region") #"us-east-1",
)
 
bedrock = boto3.client(
    service_name='bedrock', 
    region_name= os.environ.get("Region")  #'us-east-1'
)
 
# Initialize Redis client

redis_host = 'ch-agent-assist-redis-cluster-disabled.tjyhst.ng.0001.use1.cache.amazonaws.com'
#os.environ.get("RedisReadUrl") # 'ch-agent-assist-redis-cluster-disabled-ro.tjyhst.ng.0001.use1.cache.amazonaws.com'

redis_port = 6379 # Default Redis port is 6379

redis_password = None  # None if no password

redis_client = redis.Redis(host=redis_host, port=redis_port, password=redis_password, decode_responses=True)
 
prompt_bucket = 'ch-agent-assist-prompt-library-bucket'

file_key = 'prompts_library.json'


insurance_statistics = '''
{
  "ANTHEM INC.": {
    "ALTSCRIPTS": "5 days",
    "RECEPTRX": "14 days",
    "GENTRY HEALTH": "15 days",
    "ALLIANCE RX": "22 days",
    "BIOPLUS": "26 days"
  },

  "BLUE CROSS/BLUE SHIELD": {
    "US BIO": "6 days",
    "RECEPTRX": "24 days",
    "ALLIANCE RX": "43 days",
    "KROGER SPECIALTY PHARMACY": "45 days",
    "ACARIA": "45 days"
  },

  "CENTENE CORPORATION": {
    "GENTRY HEALTH": "4 days",
    "CENTERWELL": "5 days",
    "FAIRVIEW SP": "5 days",
    "LUMICERA": "16 days",
    "BIOPLUS": "22 days"
  },

  "CIGNA": {
    "AMBER": "13 days",
    "CENTURY SPECIALTY": "14 days",
    "ALTSCRIPTS": "17 days",
    "KROGER SPECIALTY PHARMACY": "26 days",
    "LUMICERA": "30 days"
  },

  "CVS CAREMARK RX": {
    "FAIRVIEW SP": "6 days",
    "CENTURY SPECIALTY": "11 days",
    "ALTSCRIPTS": "14 days",
    "AMBER": "17 days",
    "ELIXIR": "28 days"
  },

  "CVS HEALTH (AETNA)": {
    "CENTURY SPECIALTY": "11 days",
    "ALTSCRIPTS": "12 days",
    "ELIXIR": "16 days",
    "GENTRY HEALTH": "17 days",
    "ALLIANCE RX": "28 days"
  },

  "DST PHARMACY SOLUTIONS": {
    "ARDON": "3 days"
  },

  "HUMANA INC.": {
    "LUMICERA": "8 days",
    "ALTSCRIPTS": "15 days",
    "ALLIANCE RX": "19 days",
    "FAIRVIEW SP": "20 days",
    "BIOPLUS": "21 days"
  },

  "OPTUMRX": {
    "ELIXIR": "17 days",
    "NOBLE HEALTH SERVICES": "24 days",
    "BIOPLUS": "28 days",
    "ACARIA": "30 days",
    "CENTERWELL": "32 days"
  },

  "RELAY HEALTH": {
    "RECEPTRX": "4 days"
    
  },
  
    "AETNA": {
    "CVS": "4 days",
    "WALGREENS": "2 days"
    
  },

  "UNITED HEALTH": {
    "CVS": "11 days",
    "WALGREENS": "14 days",
    "GENTRY HEALTH": "19 days",
    "LUMICERA": "19 days",
    "FAIRVIEW SP": "26 days",
    "ALLIANCE RX": "28 days"
  }
}

'''
 
entity_dict_v1 = {
    "patientFirstName": "",
    "patientMiddleInitial": "",
    "patientLastName": "",
    "dob": "",
    "age": "",
    "gender": "",
    "preferredLanguage": "",
    "streetAddress": "",
    "streetName": "",
    "city": "",
    "state": "",
    "zipCode": "",
    "email": "",
    "phone": "",
    "phoneType": "",
    "priorTherapy": "",
    "diagnosisName": "",
    "diagnosisIcd10Code": "",
    "dateOfDiagnosis": "",
    "insuranceProvider": "",
    "insuranceId": "",
    "planType": "",
    "effectiveDate": "",
    "expiryDate": "",
    "recordType": "",
    "rxBin": "",
    "rxGroup": "",
    "rxPcn": "",
    "cardHolderRelationshipWithThePatient": "",
    "cardHolderName": "",
    "cardHolderDob": "",
    "prescriberName": "",
    "specialty": "",
    "address": "",
    "facilityName": "",
    "pharmacyName": "",
    "spPhone": "",
    "spFax": ""
}

entity_list = list(entity_dict_v1.keys())

entities = ""

for val in entity_list:
    entities += val + "," 

entities = entities[:len(entities)-1]
# Lambda handler to intgerate with AWS

def handler(event,context):
    start_timestamp = datetime.now()
    print(f'insights_data: handler: START: {datetime.now()}: {str(event)}')
    contact_id = json.loads(event["Records"][0]['body'])["streamConnectionId"] #event.get('conversation_id')
    #print(key_to_read)
    # Check if the key exists and read the value from Redis
    if contact_id and redis_client.exists(contact_id):
        value = redis_client.get(contact_id)
        print(value) #.split("contactId"))
    else:
        print('Key not found')
    #final_transcript = ""
    #for i in range(len(event['Records'])):
     #   final_transcript += "\n" + data_preprocessing(event['Records'][i])
    final_transcript = construct_call_conversation(value)
    #print(final_transcript)
    
    #sample_transcript = "Customer : Hello , I'm Mark."
    #prompt_enrollment = get_enrollment_prompt(prompt_bucket,file_key,'entity_extraction','enrollment_form_claude',final_transcript)
    prompt_enrollment = enrollment_prompt_generator(final_transcript,entities,today_date)
    #prompt_enrollment = enrollment_prompt_generator(sample_transcript,entities)
    entity_extraction_model_invoke_timestamp = datetime.now()
    enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
    entity_extraction_model_invoke_after_timestamp = datetime.now()
    print(enrollment_data)
    #json_data = data_postprocessing(enrollment_data)
    json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
    match = json_pattern.search(enrollment_data)
    extracted_json = match.group(0)
    enrollment_json_object = json.loads(extracted_json)
    print("Printing all the keys",list(enrollment_json_object.keys()))
    enrollment_keys = list(enrollment_json_object.keys())
    if "insuranceProvider" in enrollment_json_object.keys() and "pharmacyName" in enrollment_json_object.keys() and enrollment_json_object["insuranceProvider"] != "" and enrollment_json_object["pharmacyName"] != "":
        redis_key = str(json.loads(event["Records"][0]['body'])["streamConnectionId"]) + '::insights_data'
        if redis_client.exists(redis_key):
            print(f'redis_key:{redis_key}')
            insights_redis_data_value = redis_client.get(redis_key)
            print(f'insights_redis_data_value:{insights_redis_data_value}')
            insights_redis_data = eval(insights_redis_data_value)
            print(f'insights_redis_data : {insights_redis_data}')
        #redis_client.set(redis_key,str([{"insuranceProvider":'United Health','pharmacyName':'CVS','insights':'Insights'}]))
        #redis_client.set(redis_key,str([]))
        #insights_redis_data_value = redis_client.get(redis_key)
        #print(f'insights_redis_data_value:{insights_redis_data_value}')
        #insights_redis_data = eval(insights_redis_data_value)
        #print(f'insights_redis_data : {insights_redis_data}')
        if len(insights_redis_data) == 0:
            insurance_provider = enrollment_json_object["insuranceProvider"]
            pharmacyName = enrollment_json_object["pharmacyName"]
            #insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
            insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
            insights_model_invoke_timestamp = datetime.now()
            insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
            insights_model_invoke_after_timestamp = datetime.now()
            print("Generating Insights successfully", insights)
            sns_data = sns_data_postprocessing(event,insights)
            print("sns_data : ",sns_data)
            sns_publisher(sns_data)
            insights_redis_data.append({'insuranceProvider':enrollment_json_object['insuranceProvider'],'pharmacyName':enrollment_json_object['pharmacyName'],'insights':insights})
            print(f'new_value : {insights_redis_data}')
            redis_client.set(redis_key,str(insights_redis_data))
            return {'statusCode':200,'body':'Data Sent to SNS Sucessfully.'}
          
        for val in insights_redis_data:
            if val['insuranceProvider'] == enrollment_json_object['insuranceProvider'] and val['pharmacyName'] == enrollment_json_object['pharmacyName']:
                print(f'insights_redis_data:{insights_redis_data}')
                return {"statusCode" : 200 , "body" : "Generated Insight already present in the database."}
                print('Generated Insight already present in the database.')
            #pass
            else:
                print('Have come to else...')
                insurance_provider = enrollment_json_object["insuranceProvider"]
                pharmacyName = enrollment_json_object["pharmacyName"]
                #insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
                insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
                insights_model_invoke_timestamp = datetime.now()
                insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
                insights_model_invoke_after_timestamp = datetime.now()
                print("Generating Insights successfully", insights)
                sns_data = sns_data_postprocessing(event,insights)
                print("sns_data : ",sns_data)
                sns_publisher(sns_data)
                insights_redis_data.append({'insuranceProvider':enrollment_json_object['insuranceProvider'],'pharmacyName':enrollment_json_object['pharmacyName'],'insights':insights})
                print(f'insights_redis_data:{insights_redis_data}')
                redis_client.set(redis_key,str(insights_redis_data))
                end_timestamp = datetime.now()
                duration_till_entity_model_invoking = entity_extraction_model_invoke_timestamp - start_timestamp
                duration_of_response_of_entity_model = entity_extraction_model_invoke_after_timestamp - entity_extraction_model_invoke_timestamp
                duration_between_respone_of_entityModel_and_insightsModel = insights_model_invoke_timestamp - entity_extraction_model_invoke_after_timestamp 
                duration_of_response_of_insights_model = insights_model_invoke_after_timestamp - insights_model_invoke_timestamp
                duration_till_end = end_timestamp - insights_model_invoke_after_timestamp
                print(f'insights_data: handler: END: duration_start_to_entity_model::{duration_till_entity_model_invoking}:: duration_entity_model::{duration_of_response_of_entity_model}:: duration_entity_model_to_insights_model::{duration_between_respone_of_entityModel_and_insightsModel}:: duration_insights_model::{duration_of_response_of_insights_model}:: duration_insight_model_to_end::{duration_till_end}:: output: {json.dumps(sns_data)}')
                return {"statusCode": 200,"body": "Data Sent to SNS Sucessfully"}
        else:
            new_value_list = []
            redis_client.set(redis_key,str([])) 
            insurance_provider = enrollment_json_object["insuranceProvider"]
            pharmacyName = enrollment_json_object["pharmacyName"]
            insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
            insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
            insights_model_invoke_timestamp = datetime.now()
            insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
            insights_model_invoke_after_timestamp = datetime.now()
            print("Generating Insights successfully", insights)
            sns_data = sns_data_postprocessing(event,insights)
            print("sns_data : ",sns_data)
            sns_publisher(sns_data)
            new_value_list.append({'insuranceProvider':enrollment_json_object['insuranceProvider'],'pharmacyName':enrollment_json_object['pharmacyName'],'insights':insights})
            print(f'new_value : {new_value_list}')
            redis_client.set(redis_key,str(new_value_list))
            print('Insights Lambda Triggered.')
            end_timestamp = datetime.now()
            duration_till_entity_model_invoking = entity_extraction_model_invoke_timestamp - start_timestamp
            duration_of_response_of_entity_model = entity_extraction_model_invoke_after_timestamp - entity_extraction_model_invoke_timestamp
            duration_between_respone_of_entityModel_and_insightsModel = insights_model_invoke_timestamp - entity_extraction_model_invoke_after_timestamp 
            duration_of_response_of_insights_model = insights_model_invoke_after_timestamp - insights_model_invoke_timestamp
            duration_till_end = end_timestamp - insights_model_invoke_after_timestamp
            print(f'insights_data: handler: END: duration_start_to_entity_model::{duration_till_entity_model_invoking}:: duration_entity_model::{duration_of_response_of_entity_model}:: duration_entity_model_to_insights_model::{duration_between_respone_of_entityModel_and_insightsModel}:: duration_insights_model::{duration_of_response_of_insights_model}:: duration_insight_model_to_end::{duration_till_end}:: output: {json.dumps(sns_data)}')
            return {"statusCode": 200,"body": "Data Sent to SNS Sucessfully"}
    else:
        #sns_data = sns_data_postprocessing(event," ")
        print("No Insigts Generated.")
        return {"statusCode": 200,"body": "No Insigts Generated."}
    #SQS_Publisher(sqs_data)
    #print("sns_data : ",sns_data)

In [55]:
def insights_prompt_generator(insurance_provider,pharmacy_name,insurance_statistics):
    prompt_claude = """Human: 
    
You are Agent assist tracking the Patient and agent conversation and help the agent recommend meaningful insights based on the 
insurance and pharmacy details related insights like for example suggesting which pharmacy to select based on the
patient's Insurance provider and the pharmacy using metrics like how soon the other pharmacies if having a quicker dispense time can be helpful.
The lesser the number of days to dispense the medication,the higher are the chances of recommendation of that pharmacy.

If \" """ + insurance_provider + """ \"  or \" """ + pharmacy_name + """ \" is not present in \" """ + insurance_statistics + """ \" , then just provide a response telling that you don't have sufficient information to provide any insights and do not follow any of the information in rest of this prompt.

The patients's insurance provider is  \" """ + insurance_provider + """ \" and pharmacy is  \" """ + pharmacy_name + """ \" and use the following insurance statistics data to provide the insights:
\" """ + insurance_statistics + """ \".

In the insurance statistics data provided above,the keys represent the insurance provider and the value represents the pharmacy company and the number of days
it takes to dispense the medication to the patient.

Only provide the response when you have the values for both insurance provider and pharmacy name from the patient.Do not generate any insights unless you have both these information.
Also do not recommend any insights if the patient already has the pharmacy which is the quickest according to the insurance statistics and just suggest the patient to continue
with their existing pharmacy and not change their choice of pharmacy.


Please Provide the recommendation in brief without missing any important details in about 6 lines in less than 1000 characters

Provide the response in a structured and easily readable format and the recommendation in italic


Assistant:
"""
    return prompt_claude

In [56]:
#Please Provide the Insight in brief without missing any important details in about 1000 .

# Prompt Testing

In [57]:
sample_data = 'Customer : My Insurance is with United Health and my preferred Pharmacy is CVS'
prompt_enrollment = enrollment_prompt_generator(sample_data,entities,date.today())
enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
match = json_pattern.search(enrollment_data)
extracted_json = match.group(0)
enrollment_json_object = json.loads(extracted_json)
insurance_provider = enrollment_json_object["insuranceProvider"]
pharmacyName = enrollment_json_object["pharmacyName"]
#insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
insights_model_invoke_timestamp = datetime.now()
insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
print(insights)


 Based on the information provided, the patient's insurance provider is United Health and their current pharmacy is CVS. According to the statistics, CVS takes 11 days to dispense medication for United Health members. Walgreens takes 14 days, which is longer than CVS. Therefore, I would recommend the following:

*The patient should continue using CVS pharmacy, as it has the shortest dispense time of 11 days compared to other pharmacies for United Health members.*


In [58]:
sample_data_1 = 'Customer : My Insurance is with Green Health and my preferred Pharmacy is CVS.'
prompt_enrollment = enrollment_prompt_generator(sample_data_1,entities,date.today())
enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
match = json_pattern.search(enrollment_data)
extracted_json = match.group(0)
enrollment_json_object = json.loads(extracted_json)
insurance_provider = enrollment_json_object["insuranceProvider"]
pharmacyName = enrollment_json_object["pharmacyName"]
#insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
insights_model_invoke_timestamp = datetime.now()
insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
print(insights)

 Unfortunately I do not have sufficient information about the patient's insurance provider and pharmacy to provide meaningful insights for this case. Let's move our conversation forward and I'll be happy to assist if more details become available.


In [53]:
sample_data_2 = 'Customer : My Insurance is with ABC Health and my preferred Pharmacy is CVS.'
prompt_enrollment = enrollment_prompt_generator(sample_data_2,entities,date.today())
enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
match = json_pattern.search(enrollment_data)
extracted_json = match.group(0)
enrollment_json_object = json.loads(extracted_json)
insurance_provider = enrollment_json_object["insuranceProvider"]
pharmacyName = enrollment_json_object["pharmacyName"]
#insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
insights_model_invoke_timestamp = datetime.now()
insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
print(insights)

 Unfortunately I do not have sufficient information in the provided insurance statistics data to give any meaningful insights for this patient. The patient's insurance provider "ABC Health" and chosen pharmacy "CVS" are not present in the data. Without having dispensing time information for this specific insurance provider and pharmacy combination, I cannot make an informed recommendation. I suggest we move forward with the patient's current pharmacy selection. Please let me know if any additional relevant information becomes available.


In [1]:
sample_data_3 = 'Customer : My Insurance is with United Health and my preferred Pharmacy is CVS.'
prompt_enrollment = enrollment_prompt_generator(sample_data_3,entities,date.today())
enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
match = json_pattern.search(enrollment_data)
extracted_json = match.group(0)
enrollment_json_object = json.loads(extracted_json)
insurance_provider = enrollment_json_object["insuranceProvider"]
pharmacyName = enrollment_json_object["pharmacyName"]
#insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
insights_model_invoke_timestamp = datetime.now()
insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
print(insights)

NameError: name 'enrollment_prompt_generator' is not defined

# Redis testing

In [20]:
sample_data = '''
Customer : My Insurance is with United Health and my preferred Pharmacy is CVS.
Customer : My Insurance is with United Health and my preferred Pharmacy is MVS.
'''